# DAS Pattern Matching Queries - Animals Dataset

This notebook demonstrates how to use `hyperon-experimental` with the Distributed Atomspace (DAS) to perform pattern matching queries. We use a [simple animals knowledge base](https://raw.githubusercontent.com/singnet/das/refs/heads/master/src/tests/integration/cpp/data/animal_expressions.metta) as example.

## Setup

Initialize `hyperon` MeTTa environment and create a helper function to run MeTTa programs.


In [1]:
import hyperon

metta = hyperon.MeTTa()
def run(program='!(+ 1 2)'):
    for result in metta.run(program):
        for child in result:
            print(child)

## Import DAS Module

Import the DAS module into the MeTTa environment.


In [2]:
run('!(import! &self das)')

()


## Connect to DAS

Bind a DAS connection to `&das` space. The first parameter specifies a client's host and port range (47000-47999) and the second must be a known peer address (eg. Query Agent at localhost:40002).


In [3]:
run('!(bind! &das (new-das! (localhost:47000-47999) (localhost:40002)))')

()


## Query 1: Simple Pattern Match

Find all evaluations in the knowledge base. Variables `$P` (predicate) and `$C` (concept) will match any predicate and concept.


In [4]:
run('!(match &das (EVALUATION $P $C) (EVALUATION $P $C))')

(EVALUATION (PREDICATE "four_legs") (CONCEPT "rhino"))
(EVALUATION (PREDICATE "is_animal") (CONCEPT "rhino"))
(EVALUATION (PREDICATE "has_horns") (CONCEPT "triceratops"))
(EVALUATION (PREDICATE "is_animal") (CONCEPT "chimp"))
(EVALUATION (PREDICATE "is_animal") (CONCEPT "earthworm"))
(EVALUATION (PREDICATE "is_reptile") (CONCEPT "snake"))
(EVALUATION (PREDICATE "is_reptile") (CONCEPT "triceratops"))
(EVALUATION (PREDICATE "is_dinosaur") (CONCEPT "snake"))
(EVALUATION (PREDICATE "is_animal") (CONCEPT "human"))
(EVALUATION (PREDICATE "is_dinosaur") (CONCEPT "triceratops"))
(EVALUATION (PREDICATE "four_legs") (CONCEPT "triceratops"))
(EVALUATION (PREDICATE "is_animal") (CONCEPT "monkey"))
(EVALUATION (PREDICATE "is_plant") (CONCEPT "vine"))
(EVALUATION (PREDICATE "two_legs") (CONCEPT "monkey"))
(EVALUATION (PREDICATE "no_legs") (CONCEPT "earthworm"))
(EVALUATION (PREDICATE "is_mammal") (CONCEPT "human"))
(EVALUATION (PREDICATE "is_mammal") (CONCEPT "rhino"))
(EVALUATION (PREDICATE "is_pla

## Query 2: OR Pattern

Find concepts that are either animals OR have two legs. This demonstrates disjunctive queries.


In [5]:
run('!(match &das (or (EVALUATION (PREDICATE "is_animal") (CONCEPT $C)) (EVALUATION (PREDICATE "two_legs") (CONCEPT $C))) $C)')

"human"
"monkey"
"snake"
"chimp"
"ent"
"triceratops"
"earthworm"
"rhino"


## Query 3: AND Pattern

Find concepts that are both animals AND plants. This query should return no results as nothing can be both.


In [6]:
run('!(match &das (and (EVALUATION (PREDICATE "is_animal") (CONCEPT $C)) (EVALUATION (PREDICATE "is_plant") (CONCEPT $C))) $C)')

"ent"


## Query 4: Nested AND Pattern

Complex nested query to find concepts that are: animals AND reptiles AND dinosaurs AND have horns AND have four legs. Should match triceratops.


In [7]:
run('''
  !(match &das 
    (and
        (and 
            (EVALUATION (PREDICATE "is_animal") (CONCEPT $C)) 
            (EVALUATION (PREDICATE "is_reptile") (CONCEPT $C))
        )
        (and
            (EVALUATION (PREDICATE "is_dinosaur") (CONCEPT $C))
            (and
                (EVALUATION (PREDICATE "has_horns") (CONCEPT $C))
                (EVALUATION (PREDICATE "four_legs") (CONCEPT $C))
            )
        )
    )
    $C
  )
''')

"triceratops"


## Query 5: Reverse Lookup

Find all predicates that apply to a specific concept (triceratops). This shows all properties of triceratops.


In [8]:
run('!(match &das (EVALUATION (PREDICATE $P) (CONCEPT "triceratops")) $P)')

"has_horns"
"is_dinosaur"
"is_animal"
"four_legs"
"is_reptile"


## Query 6: Find Plants

Find all concepts that are classified as plants.


In [9]:
run('!(match &das (EVALUATION (PREDICATE "is_plant") (CONCEPT $C)) $C)')

"vine"
"ent"


## Query 7: Find Four-Legged Animals

Find all concepts that have the property "four_legs".


In [10]:
run('!(match &das (EVALUATION (PREDICATE "four_legs") (CONCEPT $C)) $C)')

"triceratops"
"rhino"


## Query 8: OR Pattern - Plants or Four-Legged

Find concepts that are either plants OR have four legs. Demonstrates combining different types of properties.


In [11]:
run('!(match &das (or (EVALUATION (PREDICATE "is_plant") (CONCEPT $C)) (EVALUATION (PREDICATE "four_legs") (CONCEPT $C))) $C)')

"vine"
"triceratops"
"ent"
"rhino"
